In [ ]:
import tensorflow as tf
import numpy as np
from skimage import data, io, filters
from mlxtend.preprocessing import one_hot
import random

# set train data path
train_list_path = 'data/images_path/train.txt'
train_data_path = 'data/images/train/%s'


class DataRead:
    def __init__(self, data_list_path, data_path, batch_size, nb_classes):
        self.data_list_path = data_list_path
        self.data_path = data_path
        self.batch_num = 0
        self.batch_size = batch_size
        self.nb_classes = nb_classes

        with open(self.data_list_path) as f:
            self.data_list = f.readlines()
        self.total_batch = int(np.floor(np.size(self.data_list) / self.batch_size))

        self.label_list = [int(d.split()[1]) for d in self.data_list]
        self.data_list = [train_data_path % d.split()[0] for d in self.data_list]

        # shuffle data
        ind_shuff = np.arange(np.size(self.data_list))
        random.shuffle(ind_shuff)
        self.data_list = [self.data_list[i] for i in ind_shuff]
        self.label_list = [self.label_list[i] for i in ind_shuff]

    def Next_Batch(self):
        batch_num = int(self.batch_num)
        batch_size = int(self.batch_size)

        data_coll = io.ImageCollection(
            self.data_list[batch_num * batch_size:(batch_num + 1) * batch_size])
        data = np.array([d for d in data_coll])

        label_coll = self.label_list[batch_num * batch_size:(batch_num + 1) * batch_size]
        label = one_hot(label_coll, self.nb_classes)

        self.batch_num = self.batch_num + 1
        if self.batch_num > self.total_batch:
            self.batch_num = 0
        return (data.reshape([-1,256*256*3])/ 255.0, label)

    def All_Data(self):
        data_coll = io.ImageCollection(
            self.data_list)
        data = np.array([d for d in data_coll])

        label_coll = self.label_list
        label = one_hot(label_coll, self.nb_classes)

        return (data.reshape([-1,256*256*3]), label)


nb_classes = 13
learning_rate = 0.001
training_epochs = 45
batch_size = 50
train = DataRead(train_list_path, train_data_path, batch_size=batch_size, nb_classes=13)

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 256*256*3])
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.get_variable("W1", shape=[256*256*3, 128*128*3],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128*128*3]))
W2 = tf.get_variable("W2", shape=[128*128*3, 32*32*3],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([32*32*3]))
W3 = tf.get_variable("W3", shape=[32*32*3, 32*32*3],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32*32*3]))
W4 = tf.get_variable("W4", shape=[32*32*3, 16*16*3],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([16*16*3]))
W5 = tf.get_variable("W5", shape=[16*16*3, 8*8*3],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([8*8*3]))
W6 = tf.get_variable("W6", shape=[8*8*3, nb_classes],initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([nb_classes]))

with tf.name_scope("layer1") as scope:
    # Hypothesis (using Relu)
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
    
with tf.name_scope("layer2") as scope:
    # Hypothesis (using Relu)
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

with tf.name_scope("layer3") as scope:
    # Hypothesis (using Relu)
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
    
with tf.name_scope("layer4") as scope:
    # Hypothesis (using Relu)
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

with tf.name_scope("layer5") as scope:
    # Hypothesis (using Relu)
    L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
    L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

with tf.name_scope("layer6") as scope:
    # Hypothesis (using softmax)
    #hypothesis = tf.nn.softmax(tf.matmul(layer2, W3) + b3)
    logits = tf.matmul(L5, W6) + b6
    
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

with tf.name_scope("prediction") as scope:
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_summ = tf.summary.scalar("accuracy", accuracy)

# train my model
sess=tf.Session()
print('Learning started. It takes sometime.')
merged_summary = tf.summary.merge_all()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter('./logs/KIST/DNN')
writer.add_graph(sess.graph)
step = 0
for epoch in range(training_epochs):
    avg_cost = 0
    avg_a = 0
    total_batch = train.total_batch

    for i in range(total_batch):
        batch_xs, batch_ys = train.Next_Batch()
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _, a,s = sess.run([cost, optimizer, accuracy,merged_summary], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_a += a / total_batch

        writer.add_summary(s, global_step=step)
        step += 1

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'accuracy =', '{:.9f}'.format(avg_a))

val_list_path = 'data/images_path/val.txt'
val_data_path = 'data/images/val/%s'
val = DataRead(val_list_path, val_data_path, batch_size=100, nb_classes=13)

val_x, val_y = val.Next_Batch()
print('Accuracy:', sess.run(accuracy, feed_dict={X: val_x, Y: val_y, keep_prob: 1}))


Learning started. It takes sometime.


In [ ]:
print(L5)

In [ ]:

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = train.total_batch    

    for i in range(total_batch):
        batch_xs, batch_ys = sess.run(train.Next_Batch())
        feed_dict = {X: batch_xs, Y: batch_ys,keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
val_list_path='data/images_path/val.txt'
val_data_path='data/images/val/%s'
val=DataRead(train_list_path,train_data_path,batch_size=batch_size,nb_classes=13)

val_x, val_y=sess.run(val.Next_Batch())
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Accuracy:', sess.run(accuracy, feed_dict={X: val_x, Y: val_y,keep_prob:1}))

In [ ]:
a=tf.train.batch([train_label],batch_size=5)

In [ ]:
256*256*3

In [ ]:
train_label

In [ ]:
with tf.Session() as sess:  
    #train_label=sess.run(train_label)
    train_data=sess.run(train_data[1:100])

In [ ]:
a,b=sess.run(train.Next_Batch())

In [ ]:
a.shape

In [ ]:
train_data[1:100]


In [ ]:
import tensorflow as tf
import numpy as np
from skimage import data, io,filters

#set train data path
train_list_path='data/images_path/train.txt'
train_data_path='data/images/train/%s'

#load train data path
with open(train_list_path) as f:
    train_data_list = f.readlines()    
#load train data&label with path


for i,tdl in enumerate(train_data_list):
    if i==0:
        train_data=io.imread(train_data_path%tdl.split()[0]).reshape([-1,256*256*3])
        train_label=np.array([int(tdl.split()[1])])
    elif i<100:
        tmp=io.imread(train_data_path%tdl.split()[0]).reshape([-1,256*256*3])
        train_data=tf.concat([train_data,tmp],0)
        
        tmp=np.array([int(tdl.split()[1])])
        train_label=tf.concat([train_label,tmp],0)

nb_classes = 13
numb_of_neurons=98304
learning_rate=0.001
#convert train_label into one hot encoding
train_label=tf.one_hot(train_label,nb_classes,axis=1)

with tf.Session() as sess:  
    train_label=sess.run(train_label)
    train_data=sess.run(train_data)


X = tf.placeholder(tf.float32, [None, 256*256*3])# MNIST data image of shape 28 * 28 = 784
Y = tf.placeholder(tf.float32, [None, nb_classes])# 0 - 9 digits recognition = 10 classes
keep_prob = tf.placeholder(tf.float32)



W1 = tf.get_variable("W1", shape=[256*256*3, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([numb_of_neurons]))
W2 = tf.get_variable("W2", shape=[numb_of_neurons, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([numb_of_neurons]))
W3 = tf.get_variable("W3", shape=[numb_of_neurons, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([numb_of_neurons]))
W4 = tf.get_variable("W4", shape=[numb_of_neurons, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([numb_of_neurons]))
W5 = tf.get_variable("W5", shape=[numb_of_neurons, nb_classes],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([nb_classes]))


with tf.name_scope("layer1") as scope:
    # Hypothesis (using Relu)
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
    
with tf.name_scope("layer2") as scope:
    # Hypothesis (using Relu)
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

with tf.name_scope("layer3") as scope:
    # Hypothesis (using Relu)
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
    
with tf.name_scope("layer4") as scope:
    # Hypothesis (using Relu)
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

with tf.name_scope("layer5") as scope:
    # Hypothesis (using softmax)
    #hypothesis = tf.nn.softmax(tf.matmul(layer2, W3) + b3)
    hypothesis = tf.matmul(L4, W5) + b5
    

with tf.name_scope("cost") as scope:
    #cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    cost_summ=tf.summary.scalar("cost",cost)
    
with tf.name_scope("train") as scope:    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(cost)
    #GradientDescent doesn't work well.. I don't why Adam is better than Gradient.
is_correct= tf.equal(tf.arg_max(hypothesis,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))

training_epochs=15
batch_size=500

In [ ]:
import tensorflow as tf
import numpy as np
from skimage import data, io, filters

# set train data path
train_list_path = 'data/images_path/train.txt'
train_data_path = 'data/images/train/%s'


class DataRead:
    def __init__(self, data_list_path, data_path, batch_size, nb_classes):
        self.data_list_path = data_list_path
        self.data_path = data_path
        self.batch_num = 0
        self.batch_size = batch_size
        self.nb_classes = nb_classes

        with open(self.data_list_path) as f:
            data_list = f.readlines()
        self.total_batch = int(np.floor(np.size(data_list) / self.batch_size))

        with open(self.data_list_path) as f:
            self.data_list = f.readlines()


    def Next_Batch(self):
        first = 1
        batch_num = self.batch_num
        batch_size = self.batch_size

        for i, tdl in enumerate(self.data_list):
            if batch_num * batch_size <= i < (batch_num + 1) * batch_size:
                if first:
                    data = io.imread(self.data_path % tdl.split()[0]).reshape([-1, 256 * 256 * 3])
                    label = np.array([int(tdl.split()[1])])
                    first = 0
                else:
                    tmp = io.imread(self.data_path % tdl.split()[0]).reshape([-1, 256 * 256 * 3])
                    data = tf.concat([data, tmp], 0)

                    tmp = np.array([int(tdl.split()[1])])
                    label = tf.concat([label, tmp], 0)
        label = tf.one_hot(label, 13, axis=1)
        self.batch_num = self.batch_num + 1
        if self.batch_num > self.total_batch:
            self.batch_num = 0
        return (data, label)


nb_classes = 13
numb_of_neurons = 98304
learning_rate = 0.001
training_epochs = 15
batch_size = 30
train = DataRead(train_list_path, train_data_path, batch_size=batch_size, nb_classes=13)

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 256 * 256 * 3])
X_img = tf.reshape(X, [-1, 256, 256, 3])
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.001))
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.001))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.001))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.reshape(L3, [-1, 32 * 32 * 128])
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[32 * 32 * 128, 625], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[625, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([nb_classes]))
logits = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = train.total_batch

    for i in range(total_batch):
        batch_xs, batch_ys = sess.run(train.Next_Batch())
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))


val_list_path='data/images_path/val.txt'
val_data_path='data/images/val/%s'
val=DataRead(train_list_path,train_data_path,batch_size=100,nb_classes=13)

val_x, val_y=sess.run(val.Next_Batch())
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Accuracy:', sess.run(accuracy, feed_dict={X: val_x, Y: val_y,keep_prob:1}))


In [ ]:
import tensorflow as tf
import numpy as np
from skimage import data, io, filters
from mlxtend.preprocessing import one_hot
import random

# set train data path
train_list_path = 'data/images_path/train.txt'
train_data_path = 'data/images/train/%s'


class DataRead:
    def __init__(self, data_list_path, data_path, batch_size, nb_classes):
        self.data_list_path = data_list_path
        self.data_path = data_path
        self.batch_num = 0
        self.batch_size = batch_size
        self.nb_classes = nb_classes

        with open(self.data_list_path) as f:
            self.data_list = f.readlines()
        self.total_batch = int(np.floor(np.size(self.data_list) / self.batch_size))

        self.label_list = [int(d.split()[1]) for d in self.data_list]
        self.data_list = [train_data_path % d.split()[0] for d in self.data_list]
        
        #shuffle data
        ind_shuff=np.arange(np.size(self.data_list))
        random.shuffle(ind_shuff)
        self.data_list=[self.data_list[i] for i in ind_shuff]
        self.label_list=[self.label_list[i] for i in ind_shuff]

    def Next_Batch(self):
        batch_num = int(self.batch_num)
        batch_size = int(self.batch_size)

        data_coll = io.ImageCollection(
            self.data_list[batch_num * batch_size:(batch_num + 1) * batch_size])
        data = np.array([d for d in data_coll])

        label_coll = self.label_list[batch_num * batch_size:(batch_num + 1) * batch_size]
        label = one_hot(label_coll, self.nb_classes)

        self.batch_num = self.batch_num + 1
        if self.batch_num > self.total_batch:
            self.batch_num = 0
        return (data/255.0, label)

    def All_Data(self):
        data_coll = io.ImageCollection(
            self.data_list)
        data = np.array([d for d in data_coll])

        label_coll = self.label_list
        label = one_hot(label_coll, self.nb_classes)

        return (data/255.0, label)


nb_classes = 13
learning_rate = 0.001
training_epochs = 15
batch_size = 512
train = DataRead(train_list_path, train_data_path, batch_size=batch_size, nb_classes=13)

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 256,256,3])
X_img = X
Y = tf.placeholder(tf.float32, [None, nb_classes])
W1=tf.Variable(tf.random_normal([11,11,3,96],stddev=0.001))
L1=tf.nn.conv2d(X_img,W1,strides=[1,4,4,1],padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
L1=tf.nn.dropout(L1,keep_prob=keep_prob)


W2=tf.Variable(tf.random_normal([5,5,96,256],stddev=0.001))
L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
L2=tf.nn.dropout(L2,keep_prob=keep_prob)

W3=tf.Variable(tf.random_normal([3,3,256,384],stddev=0.001))
L3=tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
L3=tf.nn.relu(L3)
L3=tf.nn.dropout(L3,keep_prob=keep_prob)

W4=tf.Variable(tf.random_normal([3,3,384,384],stddev=0.001))
L4=tf.nn.conv2d(L3,W4,strides=[1,1,1,1],padding='SAME')
L4=tf.nn.relu(L4)
L4=tf.nn.dropout(L4,keep_prob=keep_prob)


W5=tf.Variable(tf.random_normal([3,3,384,256],stddev=0.001))
L5=tf.nn.conv2d(L4,W5,strides=[1,1,1,1],padding='SAME')
L5=tf.nn.relu(L5)
L5=tf.nn.max_pool(L5,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
L5=tf.reshape(L5,[-1,8*8*256])
L5=tf.nn.dropout(L5,keep_prob=keep_prob)

W6 = tf.get_variable("W6", shape=[8*8*256, 4096],initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([4096]))
L6 = tf.nn.relu(tf.matmul(L5,W6)+b6)
L6 = tf.nn.dropout(L6,keep_prob=keep_prob)

W7 = tf.get_variable("W7", shape=[4096, 1000],initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([1000]))
L7 = tf.nn.relu(tf.matmul(L6,W7)+b7)
L7 = tf.nn.dropout(L7,keep_prob=keep_prob)

W8 = tf.get_variable("W8", shape=[1000, 100],initializer=tf.contrib.layers.xavier_initializer())
b8 = tf.Variable(tf.random_normal([100]))
L8 = tf.nn.relu(tf.matmul(L7,W8)+b8)
L8 = tf.nn.dropout(L8,keep_prob=keep_prob)

W9 = tf.get_variable("W9", shape=[100, nb_classes],initializer=tf.contrib.layers.xavier_initializer())
b9 = tf.Variable(tf.random_normal([nb_classes]))
logits=tf.matmul(L8,W9)+b9

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = train.total_batch

    for i in range(total_batch):
        batch_xs, batch_ys = train.Next_Batch()
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))


val_list_path='data/images_path/val.txt'
val_data_path='data/images/val/%s'
val=DataRead(val_list_path,val_data_path,batch_size=100,nb_classes=13)

val_x, val_y=val.Next_Batch()
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Accuracy:', sess.run(accuracy, feed_dict={X: val_x, Y: val_y,keep_prob:1}))


In [ ]:
val_x, val_y=val.Next_Batch()
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Accuracy:', sess.run(accuracy, feed_dict={X: val_x, Y: val_y,keep_prob:1}))

In [ ]:
val_x, val_y=train.Next_Batch()
a=sess.run(logits, feed_dict={X: val_x, Y: val_y,keep_prob:1})

In [ ]:
a[10]

In [ ]:
x = [1, 2, 3, 4, 3, 4]


In [ ]:
train.batch_num

In [ ]:
train_list_path = 'data/images_path/train.txt'
train_data_path = 'data/images/train/%s'
with open(train_list_path) as f:
    data_list = f.readlines()
label_list = [int(d.split()[1]) for d in data_list]
data_list = [train_data_path % d.split()[0] for d in data_list]




In [ ]:
import random
ind_shuff=np.arange(np.size(data_list),dtype='int32')
random.shuffle(ind_shuff)
label_list=[data_list[i] for i in ind_shuff]
data_list=[label_list[i] for i in ind_shuff]


In [ ]:
data_list[1:5]

In [ ]:
k=[1,2 , 3,4,5,67,8,9,0,0]

In [ ]:
a=ind_shuff.tolist()

In [ ]:
label_list[a]

In [ ]:
a